# Lecture 11

We shall code the AR simulator for the probit model and then invert it using the inversion theorem.

Take a vector of systematic utilities

In [1]:
library(Matrix)
library(gurobi)
#seed = 777
#set.seed(seed)
U_y = c(0.4, 0.5, 0.2, 0.3, 0.1, 0)

Loading required package: slam


Simulate the market shares using the AR simulator

In [2]:
nbDraws = 1E6
nbY = length(U_y)
rho = 0.5

Covar =  rho * matrix(1,nbY,nbY) + (1-rho) * diag(1,nbY)
E = eigen(Covar)
V = E$values 
Q = E$vectors
SqrtCovar = Q%*%diag(sqrt(V))%*%t(Q) 
epsilon_iy = matrix(rnorm(nbDraws*nbY),ncol=nbY) %*% SqrtCovar
#
u_iy = t(t(epsilon_iy)+U_y) 
ui = apply(X = u_iy, MARGIN  = 1, FUN = max)
s_y = apply(X = u_iy - ui, MARGIN = 2,FUN = function(v) (length(which(v==0)))) / nbDraws

To invert the market share, simply run the optimal assignment problem

In [3]:
A1 = kronecker(matrix(1,1,nbY),sparseMatrix(1:nbDraws,1:nbDraws))
A2 = kronecker(sparseMatrix(1:nbY,1:nbY),matrix(1,1,nbDraws))
A = rbind2(A1,A2)
result = gurobi(list(A=A,obj=c(epsilon_iy),modelsense="max",rhs=c(rep(1/nbDraws,nbDraws),s_y) ,sense="="), params=list(OutputFlag=0)) 
Uhat_y = -result$pi[(1+nbDraws):(nbY+nbDraws)] + result$pi[(nbY+nbDraws)]

print("U_y (true and recovered)")
print(U_y)
print(Uhat_y)

[1] "U_y (true and recovered)"
[1] 0.4 0.5 0.2 0.3 0.1 0.0
[1] 0.4000014 0.5000061 0.2000109 0.3000013 0.1000017 0.0000000


## BLP Data

Let's take a look through the data of the Berry, Levinsohn and Pakes (1995), often referred to as BLP.

This data comes from the High Dimensional Metrics library (hdm) library. Instead of installing the package and retreiving the data (which takes a while because we need to compile these of the library), we will instead just load it up directly

In [4]:
load("BLPdata.Rdata")
BLPData
data = as.matrix(BLPData)

model.name,model.id,firm.id,cdid,id,price,mpd,air,mpg,space,hpwt,trend,share,outshr,y
AMGREM,23,15,1,129,-6.8256171,1.888146,0,1.697,1.1502,0.5289969,0,0.0010512928,0.8801063,0.82036558
AMHORN,24,15,1,130,-6.2453701,1.935989,0,1.740,1.2780,0.4943244,0,0.0006700762,0.8801063,0.36998106
AMJAVL,25,15,1,132,-4.6527775,1.716799,0,1.543,1.4592,0.4676134,0,0.0003405273,0.8801063,-0.30691499
AMMATA,26,15,1,134,-4.9219133,1.687871,0,1.517,1.6068,0.4265403,0,0.0005222419,0.8801063,0.12072050
AMAMBS,18,15,1,136,-2.8330245,1.504286,0,1.352,1.6458,0.4524887,0,0.0004424231,0.8801063,-0.04514379
BKSKYL,53,19,1,138,-4.6083331,1.726813,0,1.552,1.6224,0.4508706,0,0.0027560592,0.8801063,1.78414673
BKLSAB,45,19,1,141,-1.9046294,1.727926,0,1.553,1.7680,0.5640020,0,0.0026505939,0.8801063,1.74512865
BKCNTU,40,19,1,143,-0.4922837,2.194121,0,1.972,1.7680,0.7313675,0,0.0004873418,0.8801063,0.05155530
BKELCT,41,19,1,144,0.3743829,2.056154,0,1.848,1.8160,0.7190139,0,0.0024151269,0.8801063,1.65209674
BKRIVE,50,19,1,145,1.2089509,1.978269,0,1.778,1.7440,0.7283237,0,0.0005513978,0.8801063,0.17504615
